In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [20]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [21]:
train_data = pd.read_csv('../data/coggle_train.csv')

In [22]:
train_data.loc[0]

id                                                 103245
text    Michel Mayor used astrometric data provided by...
cat                                             Astronomy
Name: 0, dtype: object

In [23]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
s = train_data.loc[0]['text'].translate(str.maketrans('','',string.punctuation))
tokens = word_tokenize(s)

In [24]:
filtered_sentence = [w for w in tokens if not w in stop_words] 

In [25]:
stemmed_words = []
for word in filtered_sentence:
    stemmed_words.append(ps.stem(word))

## Sklearn

In [26]:
test_data = pd.read_csv('../data/coggle_test.csv')

In [27]:
train_corpus = [x['text'] for idx,x in train_data.iterrows()]
test_corpus = [x['text'] for idx,x in test_data.iterrows()]

In [28]:
corpus = train_corpus + test_corpus

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(corpus)

In [31]:
X.shape

(41639, 142944)

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [33]:
X_tfidf.shape

(41639, 142944)

In [34]:
y_train_cat = [x['cat'] for idx,x in train_data.iterrows()]

In [35]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_train_cat)

In [36]:
train_data.shape

(33391, 3)

In [37]:
X_train_tfidf = X_tfidf[:33391]

In [38]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train_tfidf, y_train_enc, random_state=7)

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [40]:
y_pred = clf.predict(X_train_tfidf)

In [41]:
print(np.mean(y_pred == y_train))

0.17477763469198287


## Testing

In [42]:
X_test_tfidf = X_tfidf[33391:]

In [43]:
predicted = clf.predict(X_test_tfidf)

## SVM

In [44]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)

In [45]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [46]:
y_train_pred = clf.predict(X_train_tfidf)

In [47]:
print(np.mean(y_train_pred == y_train))

0.14866281333293402
